In [1]:
import sys
sys.path.append('../analysis')
from utils.all import *

DEBUG:matplotlib data path: C:\Users\heine\AppData\Roaming\Python\Python39\site-packages\matplotlib\mpl-data
DEBUG:CONFIGDIR=C:\Users\heine\.matplotlib
DEBUG:interactive is False
DEBUG:platform is win32
DEBUG:CACHEDIR=C:\Users\heine\.matplotlib
DEBUG:Using fontManager instance from C:\Users\heine\.matplotlib\fontlist-v330.json


In [2]:
# Export SimpEval_22 and Simpeval_Ext
data = load_data('annotated', batch_num=[5, 6, 7, 8, 9, 10, 11], preprocess=False)

for sent in data:
    sent['system'] = sent['system'] \
        .replace('new-wiki-2', 'simpeval-22') \
        .replace('new-wiki-1', 'simpeval-22') \
        .replace(' Writing', '-written') \
        .replace('Human ', 'Human-') \
        .replace('new-wiki-3', 'simpeval-ext')

simpeval_22, simpeval_ext = [], []
for sent in data:
    if 'simpeval-22' in sent['system']:
        simpeval_22 += [sent]
    elif 'simpeval-ext' in sent['system']:
        simpeval_ext += [sent]

INFO:Loading files: ['annotated/batch_5_anton.json', 'annotated/batch_5_ayush.json', 'annotated/batch_5_kelly.json', 'annotated/batch_5_rachel.json', 'annotated/batch_5_vinayak.json', 'annotated/batch_5_vishnesh.json', 'annotated/batch_6_anton.json', 'annotated/batch_6_ayush.json', 'annotated/batch_6_kelly.json', 'annotated/batch_6_rachel.json', 'annotated/batch_6_vinayak.json', 'annotated/batch_6_vishnesh.json', 'annotated/batch_7_anton.json', 'annotated/batch_7_ayush.json', 'annotated/batch_7_kelly.json', 'annotated/batch_7_rachel.json', 'annotated/batch_7_vinayak.json', 'annotated/batch_7_vishnesh.json', 'annotated/batch_8_anton.json', 'annotated/batch_8_ayush.json', 'annotated/batch_8_kelly.json', 'annotated/batch_8_rachel.json', 'annotated/batch_8_vinayak.json', 'annotated/batch_8_vishnesh.json', 'annotated/batch_9_anton.json', 'annotated/batch_9_ayush.json', 'annotated/batch_9_kelly.json', 'annotated/batch_9_rachel.json', 'annotated/batch_9_vinayak.json', 'annotated/batch_9_vishn

In [3]:
# Sanity check: Count the # annotations per system
def number_annotations_per_system(data):
    systems = set([s['system'] for s in data])
    for system in systems:
        print(f"{len([s for s in data if s['system'] == system])}: {system}")
number_annotations_per_system(simpeval_22)
print('')
number_annotations_per_system(simpeval_ext)

180: simpeval-22/Human-2-written
180: simpeval-22/GPT-3-few-shot
180: simpeval-22/GPT-3-zero-shot
180: simpeval-22/Human-1-written
180: simpeval-22/Muss
180: simpeval-22/T5-3B
180: simpeval-22/T5-11B

120: simpeval-ext/GPT-3-zero-shot
120: simpeval-ext/GPT-3-few-shot
120: simpeval-ext/T5-11B
120: simpeval-ext/Muss
120: simpeval-ext/T5-3B
120: simpeval-ext/Human-2-written
120: simpeval-ext/Human-1-written


In [4]:
with open(f"salsa/simpeval_22.json", "w") as f:
   json.dump(simpeval_22, f, indent=4)

with open(f"salsa/simpeval_ext.json", "w") as f:
   json.dump(simpeval_ext, f, indent=4)

In [5]:
# Export the preliminary annotations
preliminary = load_data('annotated', batch_num=[1, 2, 3, 4], preprocess=False)

for sent in preliminary:
    sent['system'] = sent['system'] \
        .replace('new_systems', 'turkcorpus') \
        .replace('new_systems', 'turkcorpus') \
        .replace('systems', 'turkcorpus')

number_annotations_per_system(preliminary)

INFO:Loading files: ['annotated/batch_1_anton.json', 'annotated/batch_1_ayush.json', 'annotated/batch_1_kelly.json', 'annotated/batch_2_vinayak.json', 'annotated/batch_2_vishnesh.json', 'annotated/batch_3_anton.json', 'annotated/batch_3_ayush.json', 'annotated/batch_3_kelly.json', 'annotated/batch_4_rachel.json', 'annotated/batch_4_vinayak.json', 'annotated/batch_4_vishnesh.json']

INFO:Found users: {'kelly', 'anton', 'vishnesh', 'vinayak', 'ayush', 'rachel'}



74: turkcorpus/T5.txt
74: turkcorpus/asset.test.simp
74: turkcorpus/asset.test.simp.second
74: turkcorpus/turk_corpus_random.txt
74: turkcorpus/con_simplification.txt


In [6]:
with open(f"salsa/turkcorpus_preliminary.json", "w") as f:
   json.dump(preliminary, f, indent=4)

### Prepare Ajudication

In [7]:
data = simpeval_22 + simpeval_ext

In [8]:
# Deletes 'annotation' or 'annotations' fields
def delete_annotations(data):
    for s in data:
        if 'annotation' in s.keys():
            del s['annotation']
        if 'annotations' in s.keys():
            del s['annotations']

# Given a user, for each sentence in interwoven,
# have the swap the sentence if their annotation in in
# the first poistion
def interweave(data, user=None):
    interwoven = {}
    for sentence_id in set([s['sentence_id'] for s in data]):
        sents = [s for s in data if s['sentence_id'] == sentence_id]

        # Order user's annotations to be first
        if user is not None:
            if user not in [s['user'] for s in sents]:
                continue
            sents = [s for s in sents if s['user'] == user] + [s for s in sents if s['user'] != user]

        sents = delete_annotations(sents)

        if len(sents) == 1:
            sents += [[], []]
        elif len(sents) == 2:
            sents += [[]]
        
        for i, s in enumerate(sents):
            if i not in interwoven:
                interwoven[i] = []
            interwoven[i] += [s]
    return interwoven

# Prepares original ajudication
def prepare_adjudiction(data, user=None, ids=None, batch_size=100):
    # If the ids are given, simply extract those ids from the data
    if ids is not None:
        interwoven = {}
        for i, id_ in enumerate(ids):
            sents = [s for s in data if s['sentence_id'] == id_]
            for j, s in enumerate(sents):
                if j not in interwoven:
                    interwoven[j] = []
                interwoven[j] += [s]
    else:
        interwoven = interweave(data, user=user)

    path = f"inspection"
    if user is not None:
        path += f"/{user}" 

    for i in interwoven.keys():
        if batch_size is not None and user is not None:
            for j in range(math.ceil(len(interwoven[i]) / batch_size)):
                end_idx = (j+1)*batch_size
                if end_idx >= len(interwoven[i]):
                    end_idx = len(interwoven[i])

                path = f"inspection/{user}/batch_{j+1}"
                if not os.path.exists(path):
                    os.makedirs(path)
                with open(f'{path}/set_{i}.json', "w") as f:
                    json.dump(interwoven[i][j*batch_size:end_idx], f, indent=4)
        else:
            if not os.path.exists(path):
                os.makedirs(path)
            with open(f'{path}/set_{i}.json', "w") as f:
                json.dump(interwoven[i], f, indent=4)

In [9]:
# If you are reading this code: lord have mercy on your soul
# 2 design ideas: 
    # (1) no annotator fixes spans on an annotations they have seen
    # (2) we remove anton and kelly as annotators

annotated_sent_id_by_user = {}
for sentence_id in set([s['sentence_id'] for s in data]):
    sents = [s for s in data if s['sentence_id'] == sentence_id]

    for s in sents:
        if s['user'] not in annotated_sent_id_by_user:
            annotated_sent_id_by_user[s['user']] = []
        annotated_sent_id_by_user[s['user']] += [s['sentence_id']]

sent_id_by_user = {}
for sentence_id in set([s['sentence_id'] for s in data]):
    sent_id_by_user[sentence_id] = []
    for user in annotated_sent_id_by_user.keys():
        if sentence_id in annotated_sent_id_by_user[user]:
            sent_id_by_user[sentence_id] += [user]

not_annotated_sent_id_by_user = {}
for user in annotated_sent_id_by_user.keys():
    new_list = []
    for sentence_id in set([s['sentence_id'] for s in data]):
        if sentence_id not in annotated_sent_id_by_user[user]:
            new_list += [sentence_id]
    not_annotated_sent_id_by_user[user] = new_list

del not_annotated_sent_id_by_user['anton']
del not_annotated_sent_id_by_user['kelly']

import random

picked_users = {}
for user in not_annotated_sent_id_by_user.keys():
    picked_users[user] = []
for sentence_id in set([s['sentence_id'] for s in data]):
    candidate_users = [u for u in not_annotated_sent_id_by_user.keys() if sentence_id in not_annotated_sent_id_by_user[u]]
    candidate_users = [u for u in candidate_users if u in not_annotated_sent_id_by_user.keys()]
    if len(candidate_users) == 0:
        raise Exception(f"No candidate users for sentence {sentence_id}")
    # user = random.choice(candidate_users)
    user = min(candidate_users, key=lambda u: len(picked_users[u]))
    picked_users[user] += [sentence_id]
    
for user in picked_users.keys():
    print(f"{user}: {len(picked_users[user])}")

sentence_ids_by_user = picked_users

vinayak: 170
ayush: 187
rachel: 172
vishnesh: 171


In [10]:
# All annotators annotated ~350 sentences
# Assign one of those sentences to each annotator - around 117 sentences per annotator. Split into 2 batches
interwoven = {}
for user in sentence_ids_by_user.keys():
    interwoven[user] = prepare_adjudiction(data, user=user, ids=sentence_ids_by_user[user])

In [11]:
# Old assignment: Annotators see their own annotations
# data = simpeval_22 + simpeval_ext
# interwoven = {}
# for user in set([s['user'] for s in data]):
#     interwoven[user] = interweave(data, user=user)

# Old assignment: Separate each of the three annotations of each sentence
# data = simpeval_22 + simpeval_ext
# prepare_adjudiction(data)
# for user in set([s['user'] for s in data]):
#     prepare_adjudiction(data, user)